In [ ]:
%%capture
%run ./test_slovo_five.ipynb

In [ ]:
#Save for use in other notebooks:
%store -r act_features
%store -r act_test_features
%store -r labels_train
%store -r labels_test
%store -r dc_features
%store -r dc_test_features
act_features = act_features
act_test_features = act_test_features
labels_train = labels_train -1
labels_test  = labels_test - 1
dc_features = dc_features
dc_test_features = dc_test_features

In [ ]:
# import necessary packages
import os
import random
import torch
import requests, zipfile, sys
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)


In [ ]:

combined_features = np.concatenate((act_features,dc_features),axis=1)

combined_test_features = np.concatenate((act_test_features,dc_test_features),axis=1)

In [ ]:
# write the customer dataset based on the dataset
class ADataset(Dataset):
    """Fashion Mnist dataset"""

    def __init__(self, data, labels):
        self.data = torch.tensor(data)#.cuda()
        self.labels = torch.tensor(labels)#.cuda()

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        record, label = self.data[index], self.labels[index]
        return record, label


g = torch.Generator()
g.manual_seed(0)

train_set = ADataset(combined_features,labels_train)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4,generator=g)
test_set = ADataset(combined_test_features,labels_test)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False, num_workers=4,generator=g)

In [ ]:
class Network(nn.Module): #TODO fix network
    def __init__(self, num_classes):
        super().__init__()
        # define layers
        self.fully_connected = nn.Sequential(
            nn.Linear(74, 200,dtype=torch.float64),
            nn.LeakyReLU(),
            nn.Linear(200, 100,dtype=torch.float64),
            nn.LeakyReLU(),
            nn.Linear(100, 50,dtype=torch.float64),
            nn.LeakyReLU(),
            nn.Linear(50, 7,dtype=torch.float64),
        )

    def forward(self, x):
        x = self.fully_connected(x)
        return x

In [ ]:
# training function
import time
import matplotlib.pyplot as plt
from sklearn import metrics

def train(net, train_loader):
    start_time = time.perf_counter()
    loss_function = torch.nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(net.parameters(),lr=0.001, weight_decay=np.exp(-6))
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[50,60,80,100,120,200,240],gamma=0.5)

    use_cuda = torch.cuda.is_available()
    if use_cuda:
        net = net.cuda()
    print("Training start:")
    epoches = 150
    train_acc_list = []
    eval_acc_list = []
    train_loss_list = []
    for epoch in range(epoches):

        train_correct = 0.0
        train_num_images = 0.0
        eval_correct = 0.0
        eval_num_images = 0.0

        for i_batch, (images, labels) in enumerate(train_loader):
            if use_cuda:
                images = images.cuda()
                labels = labels.cuda()
            if i_batch < 7 :
                net.train()
                with torch.set_grad_enabled(True):
                    f_train = net(images)
                    loss = loss_function(f_train,labels)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                # train accuracy:
                _, preds = f_train.max(1)
                train_correct += preds.eq(labels).sum().cpu().item()
                train_num_images += len(labels)
            else:
                #eval time
                net.eval()
                with torch.no_grad():
                    outs = net(images)
                    _, preds = outs.max(1)
                    eval_correct += preds.eq(labels).sum().cpu().item()
                    eval_num_images += len(labels)

        train_acc = train_correct / train_num_images
        eval_acc = eval_correct / eval_num_images
        train_acc_list.append(train_acc)
        eval_acc_list.append(eval_acc)
        train_loss_list.append(loss.item())
        print('epoch: %d, lr: %f, accuracy: %f, loss: %f, valid accuracy: %f' % (epoch, optimizer.param_groups[0]['lr'], train_acc, loss.item(), eval_acc))
        scheduler.step()
    print(f"Training time {time.perf_counter() - start_time}")

    plt.figure(figsize=(15,5))
    plt.title("Training Log")
    plt.xlabel('epoch')
    plt.ylabel('acc')
    plt.plot(list(range(0,epoches)),train_acc_list,color='blue', label='train acc')
    plt.plot(list(range(0,epoches)),eval_acc_list,color='orange', label='train acc')
    plt.legend()
    plt.show()

    plt.figure(figsize=(15,5))
    plt.title("Training Loss")
    plt.xlabel('loss')
    plt.ylabel('acc')
    plt.plot(list(range(0,epoches)),train_loss_list,color='green', label='train loss')
    plt.legend()
    plt.show()

    return net

def eval(net, eval_loader):
    net.eval()
    correct = 0.0
    num_images = 0.0
    all_preds = []
    all_labels = []
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        net = net.cuda()
    for i_batch, (images, labels) in enumerate(eval_loader):
        if use_cuda:
                images = images.cuda()
                labels = labels.cuda()
        with torch.no_grad():
            outs = net(images)
            _, preds = outs.max(1)
            correct += preds.eq(labels).sum().cpu().item()
            num_images += len(labels)
            all_preds.extend(preds.cpu())
            all_labels.extend(labels.cpu())
    acc = correct / num_images
    matrix = metrics.confusion_matrix(all_labels, all_preds)
    return acc, all_preds, all_labels, matrix



In [ ]:
net = Network(7)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
net = train(net,train_loader)

In [ ]:
tr_acc, tr_all_preds, tr_all_labels, tr_matrix = eval(net, train_loader)
print(f" Train acc = {tr_acc}")
te_acc, te_all_preds, te_all_labels, te_matrix = eval(net, test_loader)
print(f" Eval acc = {te_acc}")

In [ ]:
train_f1 = metrics.f1_score(tr_all_labels,tr_all_preds,average='macro')
test_f1 = metrics.f1_score(te_all_labels,te_all_preds,average='macro')
print("Training data F1 score = ", train_f1)
print("Testing data F1 score = ", test_f1)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].title.set_text("Training data confusion matrix:")
ev_mat_disp = metrics.ConfusionMatrixDisplay(tr_matrix)
ev_mat_disp.plot(ax=ax[0])
ax[1].title.set_text("Testing data confusion matrix:")
ts_mat_disp = metrics.ConfusionMatrixDisplay(te_matrix)
ts_mat_disp.plot(ax=ax[1])
plt.plot()

In [ ]:
# aino hnmjykiuddddddddsdfryjuhccfsth hhfvbgjkijkggggggaaaaaaaaaaaaaaaaaaahhhhhhhhhhhhhhhhhhjjjjjjjjjjbbbbbbbbbbbbbggggggggffffffdddddaaaappppppppptyfhtthngtyhugyhrtl hppgs aruf kot relationships hahaha ggtggggg   ghmjkljhgffjgnnnnnnnnnnnnnmnnnnnnnnnnnnhh,kmlmkjhknkjhkliklhij;io;uui[p
# hgfdvrelationships fjgcbhjghjgjhklsrtgfvmioopphhnnxxxxxxzzzzzzzzcccccdfghdrtyhumkdghnbxxxxxxxxxxzzzzzzzxxzzxzxzzxzxzxzxzxzxzxzxczxzxzmbzxxzxzzzxxlkjhgfdsazxvcbnmkjhgfd